In [1]:
execution_mode = 'full'
factor = 0.1
exactDate_mode = 'xor'

# Feature Matrix Generation

In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), Swissbib's goldstandard data has been processed to form records of pairs of duplicate and pairs of unique records. These records are the starting point for the final feature matrix generation and that is the reason, why the DataFrame was called feature base. As described in [[JudACaps](./A_References.ipynb#judacaps)], the next step will be an attribute-wise comparison of each attribute pair of each record in the original feature base. This comparison will generate similarity values for each attribute pair. It will halve the number of attributes replacing each attribute pair with one value expressing their degree of similarity. The goal of this chapter is a DataFrame with the full and final feature attributes. The values of these feature attributes will be used for training and performance testing of the machine learning models in the chapters to follow.

This chapter introduces similarity metrics for string comparisons. The metrics to be used for calculating its similarity will be decided for each attribute pair of the DataFrame built in the previous chapters.

## Table of Contents

- [Data Takeover](#Data-Takeover)
- [Object Distance and Similarity](#Object-Distance-and-Similarity)
    - [Mathematical Definitions](#Mathematical-Definitions)
    - [Library TextDistance](#Library-TextDistance)
- [Similarity Metrics on Attribute Level](#Similarity-Metrics-on-Attribute-Level)
    - [Table of Contents of Attribute Similarities](#Table-of-Contents-of-Attribute-Similarities)
- [DataFrame with Attributes and Similarity Features](#DataFrame-with-Attributes-and-Similarity-Features)
- [Summary](#Summary)
    - [Full Feature Matrix with Target Vector Handover](#Full-Feature-Matrix-with-Target-Vector-Handover)

## Data Takeover

Swissbib's raw data of the goldstandard has been processed in chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb). As the first step of this chapter, this data is loaded for further processing to the feature matrix and target vector for the subsequent machine learning model chapters.

In [2]:
import os
import pandas as pd
import pickle as pk

path_goldstandard = './daten_goldstandard'

# Restore metadata so far
with open(os.path.join(path_goldstandard, 'columns_metadata.pkl'), 'rb') as handle:
    columns_metadata_dict = pk.load(handle)

# Restore results so far
df_feature_base = pd.read_pickle(os.path.join(path_goldstandard, 'feature_base_df.pkl'),
                                 compression=None)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_feature_base.columns)

df_feature_base.head()

,035liste_x,035liste_y,century_x,century_y,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,coordinate_x,coordinate_y,corporate_110_x,corporate_110_y,corporate_710_x,corporate_710_y,corporate_full_x,corporate_full_y,decade_x,decade_y,docid_x,docid_y,doi_x,doi_y,duplicates,edition_x,edition_y,exactDate_x,exactDate_y,format_postfix_x,format_postfix_y,format_prefix_x,format_prefix_y,isbn_x,isbn_y,ismn_x,ismn_y,masters_docid,musicid_x,musicid_y,pages_x,pages_y,part_x,part_y,person_100_x,person_100_y,person_245c_x,person_245c_y,person_700_x,person_700_y,pubinit_x,pubinit_y,pubword_x,pubword_y,pubyear_x,pubyear_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,ttlpart_x,ttlpart_y,volumes_x,volumes_y
0,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem englischen übersetzt von...,"grawechristian, graweursula","grawechristian, graweursula",reclam jun.,reclam jun.,[Reclam jun.],[Reclam jun.],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600,600
1,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem engl. übers. von ursula ...,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman",emma,,,"{'245': ['Emma', 'Roman']}",{'245': ['Emma']},600,600
2,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (LIBIB)000315536]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,323173349,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600,600
3,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem englischen übersetzt von...,,"grawechristian, graweursula",reclam,reclam jun.,[Reclam],[Reclam jun.],2009,2009,,,emma,"emma, roman",,,{'245': ['Emma']},"{'245': ['Emma', 'Roman']}",600,600
4,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem engl. übers. von ursula ...,,,reclam,reclam,[Reclam],[Reclam],2009,2009,,,emma,emma,,,{'245': ['Emma']},{'245': ['Emma']},600,600


Chapter [Overview and Summary](./0_OverviewSummary.ipynb) assesses the predictions of the various models. One measure to analyse the results will be the resulting confusion matrix for a model prediction. The confusion matrix will reveal cases in the testing data that the model will predict opposite to the target values of the testing data. These predictions are called false positives and false negatives. To be able to analyse the original attribute values of the affected records in detail, columns $\texttt{035liste}\_\texttt{x}$, $\texttt{035liste}\_\texttt{y}$, $\texttt{docid}\_\texttt{x}$, and $\texttt{docid}\_\texttt{y}$ need to be restored. These four attributes will now be saved to a separate DataFrame to be reloaded in later chapters.

In [3]:
# Store docid's for fast identification of row pairs in model results
df_index_docids = df_feature_base[['035liste_x', '035liste_y', 'docid_x', 'docid_y']]
# The DataFrame of pairs with target information
df_feature_base = df_feature_base[columns_metadata_dict['columns_to_use']]

df_feature_base.sample(n=5)

,duplicates,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,corporate_full_x,corporate_full_y,doi_x,doi_y,edition_x,edition_y,exactDate_x,exactDate_y,format_prefix_x,format_prefix_y,format_postfix_x,format_postfix_y,isbn_x,isbn_y,ismn_x,ismn_y,musicid_x,musicid_y,part_x,part_y,person_100_x,person_100_y,person_700_x,person_700_y,person_245c_x,person_245c_y,pubinit_x,pubinit_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,volumes_x,volumes_y
101492,0,,,,,,,,,2,,1836uuuu,2015uuuu,mu,bk,010200,020000,[],[978-0-7294-1156-1],,,134,,,13,mozartwolfgang amadeus,voltaire,schikanederemanuel,venturinodiego,von w. a. mozart ; [text von emanuel schikaned...,voltaire ; [sous la dir. de diego venturino],meyer,voltaire foundation,,,"die zauberflöte, grosse oper in zwei aufzügen ...","siècle de louis xiv (iv), chapitres 13-24",,,1,453
47878,0,,,,,schweizerische gesellschaft für bildungsforschung,,,,,,19791999,200919uu,cr,mu,030600,010100,[0252-9955],"[3-7957-6954-X, 978-3-7957-6954-3]",,9790200207781,,4355,,912,,mozartwolfgang amadeus,,"schikanederemanuel, aberthermann",[hrsg. von der schweizerischen gesellschaft fü...,wolfgang amadeus mozart ; libretto by emanuel ...,"klett und balmer, universitätsverlag, universi...",,,,"bildungsforschung und bildungspraxis, schweize...","die zauberflöte, the magic flute : opera, k 620","education et recherche, educazione e ricerca",,,1 412
64432,0,,,,,,,,,,,uuuuuuuu,1940uuuu,mu,bk,010100,020000,[],[],,,422,,,2620 5,mozartwolfgang amadeus,mozartwolfgang amadeus,,"krusegeorg richard, schikanederemanuel",,von mozart ; dichtung von emanuel schikaneder,,,,,"die zauberflöte, an opera in two acts","die zauberflöte, oper in zwei aufzügen",die zauberflöte,,1,1 74
143941,0,,,,,,,,,,,1982uuuu,18971989,bk,mu,020000,010000,[],[],,,,8501,,,,mozartwolfgang amadeus,kesslersigrid,kleinmichelrichard,sigrid kessler...[et al.],von w.a. mozart ; für pianoforte zu 2 händen m...,interkantonale lehrmittelzentrale : staatliche...,,,,"bonne chance !, cours de langue française deux...","die zauberflöte, oper in 2 akten",,,256,101
66862,0,,,,,,,,,,,18uuuuuu,193uuuuu,bk,mu,020000,010100,[],[3-7618-0609-4],,,,912,,,mozartwolfgang amadeus,mozartwolfgang amadeus,schikanederemanuel,schikanederemanuel,von emanuel schickaneder ; musik von j. mozart,musik von w. a. mozart; [text] von emanuel sch...,,eulenburg,,,"die zauberflöte, oper in zwei akten : text der...","die zauberflöte, eine deutsche oper",,,32,1 412


In [4]:
print('Number of rows labelled as duplicates {:,d}'.format(len(df_feature_base[
    df_feature_base.duplicates==1])))
print('Number of rows labelled as uniques {:,d}'.format(len(df_feature_base[
    df_feature_base.duplicates==0])))
print('Total number of rows in DataFrame {:,d}'.format(df_feature_base.shape[0],
      'number of columns', df_feature_base.shape[1]))

Number of rows labelled as duplicates 1,473
Number of rows labelled as uniques 257,955
Total number of rows in DataFrame 259,428


In [5]:
print('Part of duplicates (1) on uniques (2) in units of [%]')
print(round(100*df_feature_base.duplicates.value_counts(normalize=True), 1))

Part of duplicates (1) on uniques (2) in units of [%]
0    99.4
1     0.6
Name: duplicates, dtype: float64


DataFrame feature base is the starting point used for the further processing in this chapter.

## Object Distance and Similarity

A mathematical idea of distance and similarity is needed for understanding object pair comparison. This section starts with a motivation for calculating similarities and afterwards gives a very basic definition of the two central terms, distance and similarity. The text of this section is a summary of [[Chri2012](./A_References.ipynb#chri2012)].

### Mathematical Definitions

The attributes to be used for pair comparison may contain values of poor quality. The quality originates in the way the data has been entered at the very source. Manual data entry may suffer from mistyping, automatically scanned data may suffer from insufficiencies of the scanned base material or the recognition algorithm in the optical character recognition (OCR) processing. The basic step of a deduplication process is to identify the probability of two strings of a pair to be a pair of duplicates. This is done by calculating a similarity value between the two strings compared, rather than using an exact comparison function. Based on this common similarity value for an attribute pair, their being duplicates can be decided.

The term similarity is strongly coupled to the term of distance of two values of an attribute. Mathematically, a distance can be explained with the help of a distance function. A _distance function_ or _distance metric_ $dist(o_i, o_j)$ between two points or data objects $o_i$ and $o_j$ must fulfill four requirements.

1. $dist(o_i, o_i)=0$, the distance from an object to itself is zero.
- $dist(o_i, o_j)\ge 0$, the distance between two objects is a non-negative number.
- $dist(o_i, o_j)=dist(o_j, o_i)$, the distance between two objects is symmetric.
- $dist(o_i, o_j)\le dist(o_i, o_k)+dist(o_k, o_j)$, the triangular inequality must hold. It states that the direct distance beween two objects is never larger than the combined distance when going through a third object.

A distance value expresses the dissimilarity $d$ of two objects [[HanK2012](./A_References.ipynb#hank2012)] and can therefore be converted into a similarity value $s$, calculating $s = \frac{1}{d}$, assuming $d\gt 0$. Alternatively, assuming the distance value is normalised $0\le d\le 1$, the similarity value can be calculated to $s = 1-d$. A _similarity function_ $sim(a_i, aj)$ between two attributes which can be strings, numbers, dates, geographic locations, text, XML documents, etc. fulfills the general requirements.

1. $sim(a_i, a_i)=1$, the result of comparing a value with itself is an exact similarity.
- $sim(a_i, a_j)=0$, the similarity of values that are completely different from each other is 0. What accounts for 'complete different' depends upon the type of data that are compared.
- $0\lt sim(a_i, a_j)\lt 1$, an approximate similarity between exact similarity and total dissimilarity is calculated if two attribute values are somewhat similar to each other. What accounts for 'somewhat different' depends upon the type of data that are compared.

The dissimilarity between two objects $o_i$ and $o_j$ can be computed based on the ratio of mismatches,
$$
d(o_i, o_j) = \frac{p-m}{p},
$$
where $m$ is the number of matching attributes and $p$ is the total number of attributes describing the objects [[HanK2012](./A_References.ipynb#hank2012)]. Thus the similarity between two objects can be computed as
$$
sim(o_i, o_j) = 1 - d(o_i, o_j) = \frac{m}{p}.
$$

For data deduplication, a comparison function needs to be tailored to the type of underlying data. Although there is a correspondence between a similarity function and the mathematical concept of a distance function, not all known and implemented similarity comparison functions used for string pair comparison fulfill the requirements of a distance function. Some similarity functions are not symmetric, others do not fulfill the triangular inequality. Decision taking on the best similarity function for a string pair, will be based on the effect, a similarity function has for the purpose needed. In the case of this capstone project, this purpose is its capability to contribute to the prediction whether a pair of records is a pair duplicates or a pair of uniques.

### Library TextDistance

An internet research on string distance calculation with Python has revealed libraries [[StSi](./A_References.ipynb#stsi)], [[TeDi](./A_References.ipynb#tedi)] and seperate code snippets for distinct algorithms. After trying the referenced libraries and a downloaded code snippet for a Smith Waterman similarity [[SmWa](./A_References.ipynb#smwa)], the text distance library [[TeDi](./A_References.ipynb#tedi)] has been decided to be the best decision for this capstone project. The decision is based on the GitHub statistics of stars and the date of the latest pull requests, indicating its popularity and maintenance activity of the library. A look at the API of the library, reveals the Python library to be a complete implementation (compared to suggestions of similarity metrics in [[Chri2012](./A_References.ipynb#chri2012)]) and easy to use.

In [6]:
# Install textdistance Python library - if not done, yet.
! pip install textdistance

For using the library, see documentation in [[TeDi](./A_References.ipynb#tedi)]. For the purposes of this chapter, function $\texttt{.normalized}\_\texttt{similarity()}$ of an instantiated textdistance object will be used.

In [7]:
import textdistance as tedi

With the code line above, the library is imported for application in this chapter. In appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb) the effects of the similarity metrics of the library are compared for better understanding of their specific behaviour. This comparison for each attribute is the basis of deciding the best similarity metric available for an attribute pair.

## Similarity Metrics on Attribute Level

This section implements the decision for calculating the similarity metric for each attribute of the raw data based on appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The implementation is applied on a pair of attributes of two records, resulting in a new attribute, the similarity value, of the final feature matrix. A general function $\texttt{build_delta_feature}$ is provided by the code file [data_preparation_funcs.py](./data_preparation_funcs.py) for transforming two attributes into their feature attribute holding their similarity value.

In [8]:
import data_preparation_funcs as dpf

### Table of Contents of Attribute Similarities

- [coordinate](#coordinate)
- [corporate](#corporate)
- [doi](#doi)
- [edition](#edition)
- [exactDate](#exactDate)
- [format](#format)
- [isbn](#isbn)
- [ismn](#ismn)
- [musicid](#musicid)
- [part](#part)
- [person](#person)
- [pubinit](#pubinit)
- [scale](#scale)
- [ttlfull](#ttlfull)
- [volumes](#volumes)

In [9]:
# These dictionary attributes will be filled in function dpf.build_delta_feature()
columns_metadata_dict['similarity_metrics'] = {}
columns_metadata_dict['features'] = []

### coordinate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{coordinate}$ holds coordinates of maps. To decide whether a map covers the same geographical range, a metric will be chosen that compares the coordinate number digits from left to right. The more digits are found to be the equal, the higher the similarity value is calculated. The comparison stops with the first digit pair that differs. This algorithm is satisfyed by the LCS (Longest Common Substring comparison) algorithm and generates the wanted result, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb).

In [10]:
attribute = 'coordinate'

columns_metadata_dict['similarity_metrics'][attribute+'_E'] = tedi.LCSStr()
columns_metadata_dict['similarity_metrics'][attribute+'_N'] = tedi.LCSStr()

ne_values = ['_E', '_N']

for ne in ne_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+ne,
        columns_metadata_dict['similarity_metrics'][attribute+ne],
        columns_metadata_dict)

The length of attribute $\texttt{coordinate}$ is exactly eight digits. The distinct similarity values that may occurr form a discrete set of values with a distance of $\frac{1}{8}$ between adjacent values.

In [11]:
uniques, uniques_len = {}, {}

for ne in ne_values :
    uniques[attribute+ne], uniques_len[attribute+ne] = dpf.determine_similarity_values(
        df_feature_base, attribute+ne)

coordinate_E values range [0.    0.375 0.5   0.625 0.875 1.   ]
coordinate_N values range [0.    0.375 0.5   0.75  0.875 1.   ]


Looking at some samples of the feature matrix reveals a good match to the expectations.

In [12]:
position = 3

for ne in ne_values :
    dpf.show_samples_interval(
        df_feature_base, attribute+ne,
        uniques[attribute+ne][uniques_len[attribute+ne]-position],
        uniques[attribute+ne][uniques_len[attribute+ne]-position+1]
    )

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
69481,0,0.625,e0080855,e0080900
96166,0,0.625,e0080851,e0080900
869,1,0.875,e0080855,e0080851
862,1,0.875,e0080851,e0080855
96189,0,0.875,e0080851,e0080855


0.625 <= coordinate_E_delta <= 0.875


,duplicates,coordinate_N_delta,coordinate_N_x,coordinate_N_y
95855,0,0.750,n0460821,n0460833
54932,0,0.750,n0460826,n0460833
96190,0,0.750,n0460821,n0460833
257480,0,0.750,n0460833,n0460826
182806,0,0.875,n0460826,n0460821


0.75 <= coordinate_N_delta <= 0.875


The samples above show the wanted similarity behaviour for value ranges greater than 0. The metric has the weakness, though, that empty coordinate values, e.g. for bibliographical units other than maps, have each been calculated to a similarity of 0. Some samples for duplicates in the training data are shown below.

In [13]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute+'_E', uniques[attribute+'_E'][0], uniques[attribute+'_E'][1], 10)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
576,1,0.0,,
951,1,0.0,,
495,1,0.0,,
974,1,0.0,,
207,1,0.0,,
26,1,0.0,,
1021,1,0.0,,
157,1,0.0,,
174,1,0.0,,
419,1,0.0,,


0.0 <= coordinate_E_delta <= 0.375


This downside shall be avoided by marking pairs of missing coordinate values on both sides with a special negative value, which will point out to the models to be trained, the special case of missing information in a row. The implementation of this logic is done in function $\texttt{.mark}\_\texttt{missing}()$. The absolute value of this negative number is conrolled by a factor which is passed to the function as a parameter. The function handles explicitly two cases. The first one is missing information in both attributes (resulting in $-1*\texttt{factor}$) and the second one is missing information in only one of the two attributes (resulting in $-0.5*\texttt{factor}$).

In [14]:
for ne in ne_values :
    df_feature_base = dpf.mark_missing(df_feature_base, attribute+ne, factor)

### corporate

Attribute $\texttt{corporate}$ is a collection of corporate names. The Monge-Elkan metric compares string tokens pairwise [[Chri2012](./A_References.ipynb#chri2012)] while the LCS metric searches for the longest common substring. Assessing the differences of these two metrics with the help of their values distribution in chapter [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb), reveals a better distribution behaviour for LCS. Therefore, the LCS metric will be chosen for this attribute.

In [15]:
attribute = 'corporate_full'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#tedi.StrCmp95()
#tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [16]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

corporate_full values range [0.         0.01333333 0.01428571 0.01639344 0.01666667 0.01694915
 0.01818182 0.01960784 0.02       0.02040816 0.0212766  0.02173913
 0.02272727 0.02380952 0.025      0.02531646 0.02542373 0.02666667
 0.02702703 0.02727273 0.02857143 0.03       0.03030303 0.03125
 0.03225806 0.03278689 0.03333333 0.03389831 0.03448276 0.03571429
 0.03636364 0.03773585 0.03797468 0.03846154 0.03921569 0.04
 0.04081633 0.04166667 0.04237288 0.04255319 0.04285714 0.04347826
 0.04444444 0.04545455 0.046875   0.04761905 0.04878049 0.04918033
 0.05       0.05263158 0.05333333 0.05405405 0.05454545 0.05555556
 0.05660377 0.05714286 0.05882353 0.05932203 0.06       0.06060606
 0.06122449 0.0625     0.06329114 0.06363636 0.06382979 0.06451613
 0.06521739 0.06557377 0.06666667 0.06779661 0.06818182 0.06896552
 0.07       0.07142857 0.07272727 0.07317073 0.075      0.0754717
 0.07594937 0.07627119 0.07692308 0.078125   0.07843137 0.07894737
 0.08       0.08108108 0.08163265 0.08181818

Its $110$ part is sparsely filled and even its $710$ part comes along with a little more than $10\%$ of filling, only. The LCS metric generates a similarity of 1 for the cases where both strings of a pair are empty. Missing values on both sides may be an indicator for a pair of duplicates but due to the sparsely available information, it is a weak indicator. Therefore, the pairs with missing data on both sides of the pair, will be marked with the negative value.

In [17]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

Some sample cases are shown below for both $\texttt{corporate}$ features.

In [18]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute, 0.0, 1.0, 20
)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
1153,1,1.000000,"interkantonale lehrmittelzentrale (rapperswil,...","interkantonale lehrmittelzentrale (rapperswil,..."
222,1,0.490000,schweizerische gesellschaft für bildungsforsch...,schweizerische gesellschaft für bildungsforschung
219,1,0.490000,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforsch...
855,1,1.000000,eidgenössische landestopographie,eidgenössische landestopographie
1159,1,1.000000,philharmonia orchestra (london),philharmonia orchestra (london)
761,1,1.000000,theater st. gallen,theater st. gallen
181,1,1.000000,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforschung
430,1,1.000000,wiener philharmoniker,wiener philharmoniker
852,1,1.000000,eidgenössische landestopographie,eidgenössische landestopographie
192,1,1.000000,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforschung


0.0 <= corporate_full_delta <= 1.0


In [19]:
position = uniques_len[attribute]//2 # Let's have a look in the middle range of the similarities.

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 20)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
247306,0,0.101695,allgemeine geschichtforschende gesellschaft de...,"historischer verein des kantons solothurn, kon..."
25918,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenössisches topographisches bureau
12279,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenössisches topographisches bureau
168198,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
12240,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
47817,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
25876,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
86954,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
86957,0,0.102041,schweizerische gesellschaft für bildungsforschung,eidgenssisches topographisches bureau
200280,0,0.102041,springerlink (online service),"radio-symphonie-orchester berlin, rias kammerchor"


0.10126582278481011 <= corporate_full_delta <= 0.10204081632653061


### doi

Swissbib uses an explicit $\texttt{doi}$ attribute for its deduplication implementation. In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), the real doi identifier has been isolated with the help of a preprocessing function $\texttt{.reduce}\_\texttt{to}\_\texttt{doi}\_\texttt{element()}$, see [Data Analysis](./1_DataAnalysis.ipynb). Attribute $\texttt{doi}$ contains a single string value. The Identity metric will be used for comparing the string values of a pair in a row, calculating a similarity value of 1.0 or 0.0 for each pair. If one list is empty a value of 0 is returned.

In [20]:
attribute = 'doi'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base['doi_delta'].unique()

array([1., 0.])

Some sample cases are shown below for each category of $\texttt{doi}\_\texttt{delta}$.

In [21]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

doi values range [0. 1.]


,duplicates,doi_delta,doi_x,doi_y
248176,0,1.0,,
48389,0,1.0,,
66807,0,1.0,,
256749,0,1.0,,
200034,0,1.0,,
99194,0,1.0,,
100829,0,1.0,,
55304,0,1.0,,
157428,0,1.0,,
23337,0,1.0,,


doi_delta = 1.0


,duplicates,doi_delta,doi_x,doi_y
190373,0,0.0,10.1055/b-002-26639,
17938,0,0.0,,10.3931/e-rara-50904
150078,0,0.0,,10.5169/seals-376473
234875,0,0.0,,10.5169/seals-376396
207952,0,0.0,10.1093/cid/cir669,
167678,0,0.0,10.1093/cid/ciu795,
207986,0,0.0,10.1093/cid/cir669,
167970,0,0.0,10.1093/cid/ciu795,
125181,0,0.0,,10.5169/seals-376810
162305,0,0.0,10.1007/978-3-642-41698-9,


doi_delta = 0.0


In [22]:
# Let's have a look at some non-empty doi elements
df_doi_with_element = df_feature_base[df_feature_base.doi_x.apply(lambda x : len(x) > 0)]

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_doi_with_element, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

,duplicates,doi_delta,doi_x,doi_y
1257,1,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
1268,1,1.0,10.1093/cid/ciu795,10.1093/cid/ciu795
1470,1,1.0,10.1055/b-005-143650,10.1055/b-005-143650
1185,1,1.0,10.5167/uzh-53042,10.5167/uzh-53042
1193,1,1.0,10.1093/cid/cir669,10.1093/cid/cir669
207982,0,1.0,10.1093/cid/cir669,10.1093/cid/cir669
1259,1,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
165879,0,1.0,10.1093/cid/cir669,10.1093/cid/cir669
1252,1,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
1260,1,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9


doi_delta = 1.0


,duplicates,doi_delta,doi_x,doi_y
252970,0,0.0,10.1055/b-005-143650,
194719,0,0.0,10.5451/unibas-006503313,
207652,0,0.0,10.1093/cid/ciu795,
200517,0,0.0,10.1007/978-3-642-41698-9,
255768,0,0.0,10.1055/b-005-143650,
207490,0,0.0,10.1093/cid/ciu795,
154792,0,0.0,10.5167/uzh-53042,
167609,0,0.0,10.1093/cid/ciu795,
200403,0,0.0,10.1007/978-3-642-41698-9,
207654,0,0.0,10.1093/cid/ciu795,


doi_delta = 0.0


As can be seen above, a value of 1.0 is returned if both strings of a pair are empty. As the attribute filling of $\texttt{doi}$ is sparse, see chapter [Data Analysis](./1_DataAnalysis.ipynb), the $\texttt{doi}\_\texttt{delta}$ indicates strongly a pair of duplicates for most rows. To avoid such misleading identity indication, function $\texttt{.mark}\_\texttt{missing()}$ will be applyed to the attribute.

In [23]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### edition

In its original form in Swissbib's raw data, the edition statement is a string value which may have several words. The modelling on this attribute has been tried with and without stripping letter characters from the string. The final decision for the best processing will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb). A Jaccard similarity is tried for this attribute.

In [24]:
attribute = 'edition'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [25]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

import numpy as np

edition_delta_uniques = np.sort(df_feature_base['edition_delta'].unique())
edition_delta_uniques_len = len(edition_delta_uniques)
print('edition values range', edition_delta_uniques[:30])

edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]
edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]


The comparison results in a wide number of distinct similarity values for the goldstandard data set. Below, some examples are shown.

In [26]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position],
    edition_delta_uniques[edition_delta_uniques_len-position+2], 10)

position = edition_delta_uniques_len//2

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
139041,0,1.0,,
143038,0,1.0,,
181717,0,1.0,,
236777,0,1.0,,
143255,0,1.0,,
28528,0,1.0,,
163570,0,1.0,,
60700,0,1.0,,
113542,0,1.0,,
51997,0,1.0,,


0.6 <= edition_delta <= 1.0


,duplicates,edition_delta,edition_x,edition_y
144477,0,0.0,,1885
103559,0,0.0,,1869
22650,0,0.0,,1943
18227,0,0.0,,11
209839,0,0.0,,5
38473,0,0.0,,2
37293,0,0.0,,2
25539,0,0.0,,2
199698,0,0.0,2,
157546,0,0.0,6,


0.0 <= edition_delta <= 0.1428571428571428


,duplicates,edition_delta,edition_x,edition_y
103377,0,0.25,10,144
174727,0,0.25,6,1926
196411,0,0.25,2,1928
255282,0,0.25,7,2007
103310,0,0.20,10,1928
42310,0,0.25,1791,7
184569,0,0.25,1863,8
253001,0,0.25,7,1907
42066,0,0.25,1791,1
161681,0,0.25,6,1926


0.19999999999999996 <= edition_delta <= 0.25


Again, for $\texttt{edition}\_\texttt{delta} = 1$, many empty values of the $\texttt{edition}$ attribute can be observed. These will be marked with the special negative value in the data with the goal to distinguish them from the matching attribute pairs.

In [27]:
df_feature_base = dpf.mark_missing(df_feature_base, 'edition', factor)

In [28]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
184789,0,1.0,6,6
77009,0,1.0,3,3
109553,0,1.0,4,4
188116,0,1.0,6,6
193662,0,1.0,2,2
40271,0,1.0,2,2
182947,0,1.0,3,3
658,1,1.0,5,5
196323,0,1.0,2,2
1350,1,1.0,2,2


0.6 <= edition_delta <= 1.0


### exactDate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{exactDate}$ holds a year number stored in the first four digits. Letter 'u' is used as a placeholder for an unknown digit. The attribute may hold some month and day or a second year information in its second four digits, additionally.

The attribute will be kept as a string and will not be transformed to an integer. The feature attribute of the record pair to be compared will be calculated with a modified Hamming algorithm, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The resulting similarity will be stored in a new attribute $\texttt{exactDate}\_\texttt{delta}$ which will be taken for the model calculation.

As can be seen in chapter [Decision Tree Model](./6_DecisionTreeModel.ipynb), this attribute is important for prediction. Different ways of increasing the weight of the unknown status of a digit have been tried. The different ways can be seen in the implementations below. The algorithm decided for the final simulation will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb).

In [29]:
import string

def no_xor (x_side, y_side) :
    number = 0
    for i in range(len(x_side)) :
        if ((x_side[i] in string.ascii_lowercase) | (y_side[i] in string.ascii_lowercase)) & (x_side[i] != y_side[i]) :
            number = number + 1
    return number

print('Example comparison results in a value of', no_xor ('202a0aaa', '1920uuuu'))

Example comparison results in a value of 5


In [30]:
attribute = 'exactDate'

# Replace letter 'u' with letter 'a' for one of the two strings.
#  As an effect, the resulting Hamming similarity for a letter
#  instead of a numerical digit in either string will add with an amount 0.
df_feature_base[attribute+'_x'] = df_feature_base.exactDate_x.str.replace('u', 'a')

# Compute Hamming similarity for century string pair.
columns_metadata_dict['similarity_metrics'][attribute] = tedi.Hamming()

unknown_share = 16

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

if exactDate_mode == 'added_u':
    # Add amount of 1/16 to Hamming similarity for every letter digit.
    #  But only maximum number of letter digits in both strings of a pair.
    df_feature_base[attribute+'_delta'] = df_feature_base[[
        attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
        lambda x : x[attribute+'_delta'] + 
        max(x[attribute+'_x'].count('a'), x[attribute+'_y'].count('u'))/unknown_share, axis=1
    )
elif exactDate_mode == 'xor':
    # Add amount of 1/16 to Hamming similarity for every letter digit.
    #  But only number of position-wise xor-ed letter digits in the two strings of a pair.
    df_feature_base[attribute+'_delta'] = df_feature_base[[
        attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
        lambda x : x[attribute+'_delta'] + 
        no_xor(x[attribute+'_x'], x[attribute+'_y'])/unknown_share, axis=1
    )

In [31]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']].sample(n=10)

,exactDate_x,exactDate_y,exactDate_delta
173304,2015aaaa,15501850,0.250
230145,1900aaaa,1995uuuu,0.500
234019,2006aaaa,1869uuuu,0.250
238491,1987aaaa,1873uuuu,0.375
238797,1987aaaa,uuuuuuuu,0.500
16311,aaaaaaaa,1996uuuu,0.500
150669,aaaaaaaa,1981uuuu,0.500
116065,1900aaaa,1941uuuu,0.500
147368,2004aaaa,20071991,0.625
190202,2013aaaa,2011uuuu,0.625


All resulting values of equal strings are equal to 1.

In [32]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']][
    df_feature_base.exactDate_x == df_feature_base.exactDate_y
].sort_values('exactDate_delta', ascending=False).head()

,exactDate_x,exactDate_y,exactDate_delta
159,20022000,20022000,1.0
843,19849999,19849999,1.0
845,19969999,19969999,1.0
846,19969999,19969999,1.0
847,19969999,19969999,1.0


A discrete set of different similarity values can be found in the attribute deltas. Some sample records are shown below.

In [33]:
exactDate_deltas = np.sort(df_feature_base.exactDate_delta.unique())
exactDate_deltas

array([0.    , 0.125 , 0.25  , 0.3125, 0.375 , 0.4375, 0.5   , 0.5625,
       0.625 , 0.6875, 0.75  , 0.875 , 1.    ])

In [34]:
sample_size = 5

for i in exactDate_deltas :
    dpf.show_samples_distinct(df_feature_base, 'exactDate', i, sample_size)
    print(f'exactDate_delta = {i}')

,duplicates,exactDate_delta,exactDate_x,exactDate_y
51226,0,0.0,19849999,20130627
127698,0,0.0,19201930,20172016
207431,0,0.0,20150201,19601969
125326,0,0.0,19819999,20172016
12681,0,0.0,19791999,20130627


exactDate_delta = 0.0


,duplicates,exactDate_delta,exactDate_x,exactDate_y
181747,0,0.125,20159999,17931797
168607,0,0.125,19791999,20151475
218410,0,0.125,19819999,20012002
217434,0,0.125,19972002,18701880
175094,0,0.125,20159999,18761920


exactDate_delta = 0.125


,duplicates,exactDate_delta,exactDate_x,exactDate_y
234286,0,0.25,2006aaaa,1985uuuu
72181,0,0.25,2012aaaa,1840uuuu
113935,0,0.25,20071990,1996uuuu
220493,0,0.25,2016aaaa,1980uuuu
206732,0,0.25,2016aaaa,1985uuuu


...

exactDate_delta = 0.75


,duplicates,exactDate_delta,exactDate_x,exactDate_y
113567,0,0.875,20071990,20071991
123662,0,0.875,19829999,19819999
51535,0,0.875,19969999,19959999
51513,0,0.875,19969999,19949999
123764,0,0.875,19829999,19809999


exactDate_delta = 0.875


,duplicates,exactDate_delta,exactDate_x,exactDate_y
1013,1,1.0,17001799,17001799
217,1,1.0,19791999,19791999
89485,0,1.0,19969999,19969999
263,1,1.0,20062005,20062005
220,1,1.0,19791999,19791999


exactDate_delta = 1.0


### format

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{format}$ has been split up into two new attributes $\texttt{format}\_\texttt{prefix}$ and $\texttt{format}\_\texttt{postfix}$ which will be compared by a different similarity metrics.

- As the quality of $\texttt{format}\_\texttt{prefix}$ is expected to be high, an identity comparison should be sufficient.
- Due to the observed structure of $\texttt{format}\_\texttt{postfix}$, a q-gram based comparison will be chosen.

In [35]:
attribute = 'format'

columns_metadata_dict['similarity_metrics'][attribute+'_prefix'] = tedi.Identity()
columns_metadata_dict['similarity_metrics'][attribute+'_postfix'] = tedi.Jaccard(qval=2)

pfix_values = ['_prefix', '_postfix']

for pf in pfix_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pf,
        columns_metadata_dict['similarity_metrics'][attribute+pf],
        columns_metadata_dict)

In [36]:
for i in df_feature_base.format_prefix_delta[
    df_feature_base.format_prefix_x != df_feature_base.format_prefix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_prefix', i)
    print(f'format_prefix_delta = {i}')

,duplicates,format_prefix_delta,format_prefix_x,format_prefix_y
126493,0,0.0,cr,bk
106212,0,0.0,vm,bk
118569,0,0.0,bk,vm
31121,0,0.0,vm,bk
184898,0,0.0,bk,vm


format_prefix_delta = 0.0


In [37]:
for i in df_feature_base.format_postfix_delta[
    df_feature_base.format_postfix_x != df_feature_base.format_postfix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_postfix', i)
    print(f'format_postfix_delta = {i}')

,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
228011,0,0.428571,030000,020000
90705,0,0.428571,020000,010200
48383,0,0.428571,010100,010300
68208,0,0.428571,010200,020000
16225,0,0.428571,040100,040000


format_postfix_delta = 0.4285714285714286


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
17417,0,0.111111,040100,020000
177333,0,0.111111,030600,010053
152302,0,0.111111,010100,020000
222494,0,0.111111,020000,010300
82414,0,0.111111,040100,020000


format_postfix_delta = 0.11111111111111116


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
60041,0,0.25,020000,020353
122307,0,0.25,020000,020353
161062,0,0.25,010200,020353
68554,0,0.25,010200,020353
196623,0,0.25,010347,010200


...

format_postfix_delta = 0.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
240026,0,1.0,020000,020000
71342,0,1.0,020000,020000
145607,0,1.0,020000,020000
218634,0,1.0,020000,020000
174893,0,1.0,020000,020000


format_postfix_delta = 1.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
245199,0,0.666667,020400,020047
88975,0,0.666667,030500,030053


format_postfix_delta = 0.6666666666666666


### isbn

Swissbib uses each string element of the $\texttt{isbn}$ list separately for comparing with each string element of its comparison $\texttt{isbn}$ list. If two bibliographic units hold at least one element in common, this is interpreted as a strong indicator for duplicates [[WiCo2001](./A_References.ipynb#wico2001)].

This hard logic is used in a modified way in the context of this capstone project. A special comparison function $\texttt{.build}\_\texttt{delta}\_\texttt{isbn()}$ has been implemented that compares each list element of the left-hand side with each list element of the right-hand side of a pair. According to Swissbib's implementation, the Identity metric is used for string comparison, calculating a similarity value of 1.0 or 0.0 for each list element pair. For normalisation reasons, the sum of similarity values is divided by the number of elements of the smaller list. If both lists are empty a value of 1.0 is returned. If only one list is empty a value of 0.0 is returned.

In [38]:
attribute = 'isbn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base[attribute+'_delta'].unique()

array([1. , 0. , 0.5])

Some sample cases are shown below for each category of $\texttt{isbn_delta}$.

In [39]:
for isbn_delta_value in df_feature_base['isbn_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['isbn_delta']==isbn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'isbn', isbn_delta_value, number_of_max_samples)
    print(f'isbn_delta = {isbn_delta_value}')

,duplicates,isbn_delta,isbn_x,isbn_y
11489,0,1.0,[],[]
17039,0,1.0,[],[]
92309,0,1.0,[],[]
64859,0,1.0,[],[]
10495,0,1.0,[],[]
92392,0,1.0,[],[]
242370,0,1.0,[],[]
64710,0,1.0,[],[]
113882,0,1.0,[],[]
141355,0,1.0,[],[]


isbn_delta = 1.0


,duplicates,isbn_delta,isbn_x,isbn_y
251109,0,0.0,[],"[978-3-598-31799-6 (print), 978-3-11-096276-5]"
209527,0,0.0,[978-3-13-127286-7],[]
83317,0,0.0,[3-254-08008-4],[3-499-17476-6]
61351,0,0.0,[],"[978-3-598-31801-6 (print), 978-3-11-095538-5]"
129178,0,0.0,[],[978-3-13-127285-0]
24988,0,0.0,[3-8067-5097-1],[]
194193,0,0.0,"[3-495-48796-4, 978-3-495-48796-9]","[978-0-19-953552-1, 0-19-953552-3]"
7645,0,0.0,[],[2-08-070552-0]
2605,0,0.0,[],"[978-3-598-31482-7 (print), 978-3-11-096891-0]"
162318,0,0.0,"[978-3-642-41697-2, 978-3-642-41698-9 (ebook)]",[]


isbn_delta = 0.0


,duplicates,isbn_delta,isbn_x,isbn_y
1205,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1201,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1199,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1195,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1210,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1202,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"


isbn_delta = 0.5


For attribute $\texttt{isbn}$, the special marking of missing values is omitted.

### ismn

This attribute will be processed with the identity similarity metric. The reasoning for this decision is the same as for similar attributes above. 

In [40]:
attribute = 'ismn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()
#tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [41]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

ismn values range [0. 1.]


In [42]:
for ismn_delta_value in df_feature_base[attribute+'_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base[attribute+'_delta']==ismn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'ismn', ismn_delta_value, number_of_max_samples)
    print(f'ismn_delta = {ismn_delta_value}')

,duplicates,ismn_delta,ismn_x,ismn_y
65575,0,1.0,,
75870,0,1.0,,
182987,0,1.0,,
73442,0,1.0,,
23484,0,1.0,,
97547,0,1.0,,
214723,0,1.0,,
765,1,1.0,,
89728,0,1.0,,
142312,0,1.0,,


ismn_delta = 1.0


,duplicates,ismn_delta,ismn_x,ismn_y
250182,0,0.0,m200205343,
80018,0,0.0,m006546756,
103358,0,0.0,m006450510,
20423,0,0.0,m006450510,
191575,0,0.0,,m006450510
136949,0,0.0,m006204687,
135170,0,0.0,m006450510,
114784,0,0.0,"m006546756 (kritischer bericht, leinen)",
135364,0,0.0,m006450510,
251114,0,0.0,m006204687,


ismn_delta = 0.0


As can be seen in the previous chapters, attribute $\texttt{ismn}$ is filled sparsely. A lot of missing values calculate to a value of 1.0 in the chosen similarity metrics. To mark these cases specifically, they will be transformed to a negative value.

In [43]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### musicid

Chapter [Data Analysis](./1_DataAnalysis.ipynb) shows that attribute $\texttt{musicid}$ is an identifyer for a music record. A Jaccard metric has been tested on this attribute, resulting in a distribution of many high similarity values on uniques. Comparing this result with the LCS metric, the latter has been decided.

In [44]:
attribute = 'musicid'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [45]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

musicid values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.375      0.4        0.42857143 0.5
 0.6        0.66666667 0.75       1.        ]


In [46]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//2

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)

,duplicates,musicid_delta,musicid_x,musicid_y
585,1,1.0,99036,99036
986,1,1.0,10425,10425
424,1,1.0,0171433,0171433
722,1,1.0,422,422
507,1,1.0,4553,4553
942,1,1.0,245,245
1022,1,1.0,0630,0630
276,1,1.0,502023,502023
478,1,1.0,10425,10425
493,1,1.0,10425,10425


0.75 <= musicid_delta <= 1.0


,duplicates,musicid_delta,musicid_x,musicid_y
6574,0,0.0,,
191227,0,0.0,,
16878,0,0.0,,
155997,0,0.0,,
2145,0,0.0,,
89168,0,0.0,,
246413,0,0.0,,
251401,0,0.0,134,
135474,0,0.0,4553,
225511,0,0.0,,


0.0 <= musicid_delta <= 0.1428571428571429


,duplicates,musicid_delta,musicid_x,musicid_y
140889,0,0.4,10425,410
105418,0,0.4,422,10425
115983,0,0.4,10425,410
18246,0,0.4,10425,422
75760,0,0.4,10425,1092
68121,0,0.4,10425,8421
75705,0,0.4,10425,542
74084,0,0.4,10425,422
15728,0,0.4,422,10425
134570,0,0.4,10425,422


0.375 <= musicid_delta <= 0.4


In [47]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1], attribute,
    uniques[attribute][0],
    uniques[attribute][uniques_len[attribute]-1], 20)

,duplicates,musicid_delta,musicid_x,musicid_y
574,1,0.0,,
348,1,0.0,,
1197,1,0.0,,
303,1,1.0,502023,502023
172,1,0.0,,
1320,1,0.0,,
441,1,0.0,,
885,1,0.0,,
1107,1,0.0,,
1318,1,0.0,,


0.0 <= musicid_delta <= 1.0


The attribute is filled with a degree of below $10\%$. The chosen metric for it results in a similarity value of 1.0 for empty value pairs. This effect can be adjusted with function $\texttt{.mark}\_\texttt{missing}()$ as above. 

In [48]:
df_feature_base = dpf.mark_missing(df_feature_base, 'musicid', factor)

### part

Analogous to attribute $\texttt{edition}$ described above, the string value of this attribute can be stripped to pure number digits. Both ways, with and without letter stripping have been tried for modelling. The final decision for the best processing will be documented in chapter [Overview and Summary](./0_OverviewSummary.ipynb). Three different metrics have been tried for attribute $\texttt{part}$. Finally, metric Jaro will be used.

In [49]:
attribute = 'part'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
#tedi.Jaro()
#tedi.Hamming()
#tedi.LCSStr()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [50]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

part values range [0.         0.27407407 0.28240741 0.29202279 0.30740741 0.31318681
 0.31481481 0.32407407 0.32898551 0.33333333 0.33597884 0.33838384
 0.33921569 0.34444444 0.3452381  0.35128205 0.35185185 0.35897436
 0.36111111 0.36231884 0.36666667 0.37037037 0.37179487 0.37254902
 0.37301587 0.37407407 0.375      0.37777778 0.38333333 0.38461538
 0.38568376 0.38888889 0.38927739 0.39070048 0.39215686 0.39393939
 0.3952381  0.3960114  0.39646465 0.39722222 0.39814815 0.4
 0.4031746  0.4037037  0.40604575 0.40740741 0.40842491 0.40855763
 0.41025641 0.41111111 0.41125541 0.41176471 0.41203704 0.41282051
 0.41388889 0.41449275 0.41452991 0.41507937 0.41666667 0.4178744
 0.41798942 0.41851852 0.41919192 0.42083333 0.42222222 0.42261905
 0.42390289 0.4241453  0.42564103 0.42592593 0.42777778 0.42810458
 0.42857143 0.42948718 0.43030303 0.43055556 0.43115942 0.43174603
 0.43236715 0.43333333 0.43518519 0.43557423 0.43589744 0.43627451
 0.43650794 0.43703704 0.4375     0.43813131 0.43888

In [51]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//7

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

,duplicates,part_delta,part_x,part_y
240514,0,1.0,,
221818,0,1.0,,
47396,0,1.0,,
40082,0,1.0,,
61993,0,1.0,,
249424,0,1.0,,
9232,0,1.0,,
39579,0,1.0,,
243601,0,1.0,,
77167,0,1.0,,


0.9215686274509803 <= part_delta <= 1.0


,duplicates,part_delta,part_x,part_y
176123,0,0.0,13,
10437,0,0.0,,2002 107 117
189208,0,0.0,208,
197139,0,0.0,,20 20
112334,0,0.0,,50 50
231735,0,0.0,,3
34996,0,0.0,27 597 27 597,
153558,0,0.0,,552 552
69132,0,0.0,285,
252661,0,0.0,313 2017,


0.0 <= part_delta <= 0.28240740740740744


,duplicates,part_delta,part_x,part_y
117739,0,0.714286,552 552,2
197341,0,0.714286,23 1862,3
132414,0,0.714286,4,241 319
131797,0,0.714286,3,23 1902
214585,0,0.714286,912 912,2
227411,0,0.714286,285 285,2
257234,0,0.714286,23 1862,3
214466,0,0.714286,912 912,2
257290,0,0.714286,23 1862,3
257162,0,0.714286,23 1862,2


0.7142857142857143 <= part_delta <= 0.7146464646464646


,duplicates,part_delta,part_x,part_y
197572,0,0.717338,23 1862,23 23 1870 23
37481,0,0.716667,5 19,15 15 1926
257493,0,0.717338,23 1862,23 23 1899 23
197602,0,0.717338,23 1862,23 23 1889 23
197596,0,0.717338,23 1862,23 23 1870 23
257465,0,0.717338,23 1862,23 23 1870 23
257479,0,0.717338,23 1862,23 23 1988 23
256298,0,0.716667,5 19,15 15 1926
257489,0,0.717338,23 1862,23 23 1870 23
115164,0,0.716667,5 19,15 15 1926


0.7159544159544161 <= part_delta <= 0.7173382173382173


In this attribute, too, moving pairs of empty values to negative values will result in a clearer distinction between pairs of uniques and duplicates, as will be seen in the graphical comparison of capter [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb).

In [52]:
df_feature_base = dpf.mark_missing(df_feature_base, 'part', factor)

### person

As a result of chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{person}$ has been split into three specific attributes. Attribute $\texttt{person}\_{100}$ and $\texttt{person}\_{700}$ hold strongly standardised string values. For comparing pure strings, a Levenshtein metric is recommended [[Chri2012](./A_References.ipynb#chri2012)]. Unfortunately, this metric shows a very long calculation time on the data of the capstone project. Comparing the similarity values of the Levenshtein metric with the similarity values of other metrics in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), similarity metric StrCmp95 has been decided to use.

In [53]:
attribute = 'person'

columns_metadata_dict['similarity_metrics'][attribute+'_100'] = tedi.StrCmp95()
columns_metadata_dict['similarity_metrics'][attribute+'_700'] = tedi.StrCmp95()
#tedi.Levenshtein()

pe_values = ['_100', '_700']

for pe in pe_values :
    print('Calculating person'+pe)
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pe,
        columns_metadata_dict['similarity_metrics'][attribute+pe],
        columns_metadata_dict)

Calculating person_100


Calculating person_700


In [54]:
pe = '_100'

uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_100 values range [0.         0.31944444 0.32777778 0.33095238 0.34555556 0.34768519
 0.3562963  0.35714286 0.36414141 0.36527778 0.36723647 0.37777778
 0.38511815 0.38603989 0.38779956 0.38989899 0.39246032 0.39490741
 0.40142857 0.40347222 0.40666667 0.41281046 0.4130719  0.41596491
 0.42324786 0.42388889 0.4240404  0.42486772 0.42614379 0.42620321
 0.42676768 0.42819444 0.42869281 0.42936508 0.42982456 0.42991453
 0.43208333 0.43326118 0.43415893 0.43508772 0.43555556 0.43590643
 0.43660714 0.43794872 0.43804714 0.439819   0.44040404 0.44079254
 0.44126984 0.44222222 0.44282407 0.44292929 0.44444444 0.44457516
 0.44507937 0.44571078 0.44649123 0.44656085 0.4465812  0.44662698
 0.44722222 0.44782135 0.44786325 0.44791667 0.4481352  0.44861111
 0.44972222 0.45126263 0.45131313 0.45222222 0.45291375 0.45314327
 0.45339367 0.45416667 0.45424242 0.45555556 0.45564103 0.45578704
 0.45614973 0.45784314 0.45873016 0.45952381 0.46013072 0.46055556
 0.46119281 0.46127946 0.46336996 0.46

In [55]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_100_delta,person_100_x,person_100_y
13133,0,1.0,austenjane,austenjane
42457,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
146311,0,1.0,,
82774,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
55134,0,1.0,,
21900,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus
7932,0,1.0,,
35720,0,1.0,voltaire,voltaire
226285,0,1.0,,
82842,0,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus


0.8666666666666666 <= person_100_delta <= 1.0


,duplicates,person_100_delta,person_100_x,person_100_y
190576,0,0.0,trappehans-joachim,
95204,0,0.0,mozartwolfgang amadeus,
38226,0,0.0,,mortzfeldpeter
119508,0,0.0,,mozartwolfgang amadeus
122808,0,0.0,,bührerwalter
120019,0,0.0,,mortzfeldpeter
197152,0,0.0,,mortzfeldpeter
101885,0,0.0,mozartwolfgang amadeus,
205015,0,0.0,,steinerrudolf
41530,0,0.0,mozartwolfgang amadeus,


0.0 <= person_100_delta <= 0.3277777777777777


For comparing person names, like in attribute $\texttt{person}\_{245c}$, a Jaro metric will be tested [[Chri2012](./A_References.ipynb#chri2012)].

In [56]:
pe = '_245c'

columns_metadata_dict['similarity_metrics'][attribute+pe] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute+pe,
    columns_metadata_dict['similarity_metrics'][attribute+pe],
    columns_metadata_dict)

In [57]:
uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_245c values range [0.         0.24632035 0.25462963 ... 0.99583333 0.99603175 1.        ]


In [58]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_245c_delta,person_245c_x,person_245c_y
82507,0,1.0,wolfgang amadeus mozart,wolfgang amadeus mozart
245029,0,1.0,,
233272,0,1.0,,
58547,0,1.0,,
89345,0,1.0,,
89050,0,1.0,,
49797,0,1.0,,
71,1,1.0,andreas flury,andreas flury
210929,0,1.0,,
1249,1,1.0,"hans-joachim trappe, hans-peter schuster","hans-joachim trappe, hans-peter schuster"


0.996031746031746 <= person_245c_delta <= 1.0


,duplicates,person_245c_delta,person_245c_x,person_245c_y
233878,0,0.0,,"mortzfeld, peter; raabe, paul"
251915,0,0.0,,sigrid kessler ... <et al.> ; <éds.> interkant...
21807,0,0.0,von emanuel schikaneder ; musik von w.a. mozar...,
212286,0,0.0,,sigrid kessler [u.a.]
64491,0,0.0,,von emmanuel schikaneder; die musik ist von he...
57914,0,0.0,hrsg. von attila csampai und dietmar holland,
54112,0,0.0,[red.: ute vollmar],
89232,0,0.0,,jane austen ; edited by james kinsley ; with a...
239922,0,0.0,[von emmanuel schikaneder] ; [die musik ist vo...,
37557,0,0.0,wolfgang amadeus mozart ; [text von emanuel sc...,


0.0 <= person_245c_delta <= 0.25462962962962954


The similarities of all three $\texttt{person}$ attributes are affected by empty values. These will be handled the same way as the attributes above.

In [59]:
pe_values = ['_100', '_245c', '_700']

for pe in pe_values :
    df_feature_base = dpf.mark_missing(df_feature_base, 'person'+pe, factor)

### pubinit

This attribute holds publisher strings that have a similar representation as attribute $\texttt{person}$. A Jaro metric will be used.

In [60]:
attribute = 'pubinit'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [61]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

pubinit values range [0.         0.25132275 0.25303644 ... 0.98404165 0.98484848 1.        ]


In [62]:
position = uniques_len[attribute]//3

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-1],
    uniques[attribute][uniques_len[attribute]-position], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-5],
    uniques[attribute][uniques_len[attribute]-1], 10)

,duplicates,pubinit_delta,pubinit_x,pubinit_y
97706,0,0.554626,"tdk recording media europe, opernhaus",p. reclam jun.
97782,0,0.554626,"tdk recording media europe, opernhaus",p. reclam jun.
149371,0,0.554701,erato,"interkantonale lehrmittelzentrale ; [bern], be..."
45003,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
2900,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
149367,0,0.554701,erato,"interkantonale lehrmittelzentrale ; [bern], be..."
44999,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
90660,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
199776,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."
2896,0,0.554701,alber,"interkantonale lehrmittelzentrale ; [bern], be..."


0.5546260546260546 <= pubinit_delta <= 0.5547008547008546


,duplicates,pubinit_delta,pubinit_x,pubinit_y
207184,0,1.0,,
194345,0,1.0,,
248033,0,1.0,,
30687,0,1.0,,
79268,0,1.0,,
61723,0,1.0,,
232659,0,1.0,,
20004,0,1.0,,
169394,0,1.0,,
192518,0,1.0,,


0.9777777777777779 <= pubinit_delta <= 1.0


The similarities of $\texttt{pubinit}$ is affected by empty values. These will be transformed to negative values.

In [63]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### scale

Comparing the similarity metrics of some sample value pairs of attribute $\texttt{scale}$ in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), a Jaccard metrics has been identifyed to express the best matching behaviour for purely numerical values stored in the attribute.

In [64]:
attribute = 'scale'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [65]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

scale values range [0.         0.17857143 0.21428571 0.57142857 1.        ]


In [66]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,scale_delta,scale_x,scale_y
254519,0,0.0,,100000
7226,0,0.0,,100000
73910,0,0.0,,100000
182867,0,0.0,50000,
135903,0,0.0,,100000
2495,0,0.0,,50000
55088,0,0.0,50000,
219785,0,0.0,,100000
228416,0,0.0,,100000
180059,0,0.0,,100000


0.0 <= scale_delta <= 0.1785714285714286


,duplicates,scale_delta,scale_x,scale_y
95854,0,0.571429,50000,100000
55263,0,0.571429,50000,100000
69506,0,0.571429,50000,100000
182433,0,0.571429,50000,100000
95856,0,0.571429,50000,100000
55282,0,0.571429,50000,100000
96182,0,0.571429,50000,100000
55274,0,0.571429,50000,100000
227836,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
197581,0,0.571429,100000,50000


0.2142857142857143 <= scale_delta <= 0.5714285714285714


,duplicates,scale_delta,scale_x,scale_y
227820,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227822,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227835,0,0.178571,50 000 8 10 8 35 45 55 46 05,50000
227498,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227813,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227824,0,0.178571,50 000 8 10 8 35 45 55 46 05,50000
227496,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227844,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
227838,0,0.214286,50 000 8 10 8 35 45 55 46 05,100000
866,1,0.178571,50000,50 000 8 10 8 35 45 55 46 05


0.1785714285714286 <= scale_delta <= 0.2142857142857143


Attribute $\texttt{scale}$ is filled for maps, only. Due to its sparse filling, the similarities of the attribute are affected strongly by empty values. These empty values will be marked with a special negative value.

In [67]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

### ttlfull

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{ttlfull}$ has been split up into two new attributes $\texttt{ttlfull_245}$ and $\texttt{ttlfull_246}$ which will be compared by the same similarity metrics. A visual analysis of the values stored in the attribute, reveals a string of words, comparable to the strings in attribute $\texttt{person_245c}$, above. The same similarity metric will be used for both title attributes, therefore.

In [68]:
attribute = 'ttlfull'

columns_metadata_dict['similarity_metrics'][attribute+'_245'] = tedi.Jaro()
columns_metadata_dict['similarity_metrics'][attribute+'_246'] = tedi.Jaro()

tf_values = ['_245', '_246']

for tf in tf_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+tf,
        columns_metadata_dict['similarity_metrics'][attribute+tf],
        columns_metadata_dict)

In [69]:
for tf in tf_values :
    uniques[attribute+tf], uniques_len[attribute+tf] = dpf.determine_similarity_values(
        df_feature_base, attribute+tf)

ttlfull_245 values range [0.         0.24175824 0.24358974 ... 0.99555556 0.99747475 1.        ]
ttlfull_246 values range [0.         0.27407407 0.35555556 0.36111111 0.36366612 0.36666667
 0.39444444 0.39814815 0.40238095 0.40555556 0.41140642 0.41190476
 0.41296296 0.41313131 0.4168847  0.41923077 0.41931217 0.42380952
 0.42424242 0.42484886 0.42767857 0.43003072 0.43333333 0.43758865
 0.43913043 0.43998145 0.44198966 0.44242424 0.44507736 0.44539295
 0.44722222 0.4479837  0.44848485 0.44901497 0.45       0.45011338
 0.45277778 0.45436508 0.45610717 0.45714286 0.45764895 0.45833333
 0.46094276 0.46205962 0.46527102 0.46729958 0.46851852 0.46885522
 0.46944444 0.46945938 0.47130647 0.47142857 0.47162447 0.47222222
 0.472318   0.47267613 0.47285513 0.47308219 0.47356964 0.47372609
 0.47388167 0.47417027 0.47428613 0.47431581 0.47446677 0.47462215
 0.47491364 0.47508251 0.47515152 0.47518634 0.47597067 0.47606838
 0.47613256 0.47629884 0.47642654 0.47794389 0.47829216 0.47836071
 0.4788

In [70]:
tf = '_245'
position = uniques_len[attribute+tf]

dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-position],
    uniques[attribute+tf][uniques_len[attribute+tf]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-3],
    uniques[attribute+tf][uniques_len[attribute+tf]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-4],
    uniques[attribute+tf][uniques_len[attribute+tf]-3], 10)

,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
102635,0,0.0,emma,blick in die welt
33858,0,0.0,arts,blick in die welt
33511,0,0.0,arts,emma
78921,0,0.0,emma,blick in die welt
6381,0,0.0,bildungsforschung und bildungspraxis,emma
11912,0,0.0,emma,blick in die welt
19039,0,0.0,emma,blick in die welt
13115,0,0.0,emma,blick in die welt
219552,0,0.0,bildungsforschung und bildungspraxis,emma
102672,0,0.0,emma,switzerland


0.0 <= ttlfull_245_delta <= 0.2417582417582418


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
144730,0,0.995556,"bonne chance !, cours de langue française deux...","bonne chance !, cours de langue française, deu..."
249,1,0.997475,sozialleistungsbetrug: sozialversicherungsbetr...,sozialleistungsbetrug: sozialversicherungsbetr...
237,1,0.997475,sozialleistungsbetrug: sozialversicherungsbetr...,sozialleistungsbetrug: sozialversicherungsbetr...


0.9955555555555556 <= ttlfull_245_delta <= 0.9974747474747474


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
128572,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
1360,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
84,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
97,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
53004,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
33,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
58,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
50,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
1362,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
128908,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."


0.9953703703703703 <= ttlfull_245_delta <= 0.9955555555555556


Attribute $\texttt{ttlfull}\_\texttt{245}$ is filled for all data rows of Swissbib's raw data as can be seen in chapter [Data Analysis](./1_DataAnalysis.ipynb). For attribute $\texttt{ttlfull}\_\texttt{245}$, the filling is below $10\%$. The data pairs with missing values will be marked with a negative value as has been done for similar cases above.

In [71]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute+'_246', factor)

### volumes

This attribute is described in chapter [Data Analysis](./1_DataAnalysis.ipynb) to hold a kind of contents that resembles the contents of attribute $\texttt{part}$. The same similarity metrics will be used for attribute $\texttt{volumes}$ as for attribute $\texttt{part}$, therefore.

In [72]:
attribute = 'volumes'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
#tedi.Jaro()
#tedi.LCSSeq()
#tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [73]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

volumes values range [0.         0.31318681 0.32222222 0.32905983 0.35128205 0.37301587
 0.37407407 0.38333333 0.38461538 0.3952381  0.40740741 0.41111111
 0.41666667 0.41737892 0.42857143 0.43650794 0.43703704 0.44017094
 0.44166667 0.44200244 0.44230769 0.44761905 0.4537037  0.45555556
 0.45833333 0.46296296 0.46428571 0.46581197 0.46666667 0.47008547
 0.47222222 0.47619048 0.47777778 0.48148148 0.48333333 0.48412698
 0.48611111 0.48888889 0.49007937 0.49145299 0.49206349 0.49365079
 0.4991453  0.5        0.50793651 0.51111111 0.51190476 0.52222222
 0.52380952 0.52564103 0.52777778 0.53333333 0.53653846 0.53703704
 0.53968254 0.54074074 0.54166667 0.54304029 0.54401709 0.54722222
 0.54761905 0.5491453  0.55       0.55128205 0.55555556 0.56031746
 0.56111111 0.56190476 0.56507937 0.56630037 0.56666667 0.56944444
 0.57407407 0.57478632 0.57777778 0.58148148 0.58333333 0.58730159
 0.58862434 0.58888889 0.59444444 0.5952381  0.59722222 0.5982906
 0.6        0.60119048 0.60320513 0.606837

In [74]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,volumes_delta,volumes_x,volumes_y
135385,0,0.0,1 221,254
214146,0,0.0,388,
119360,0,0.0,134,256
97883,0,0.0,2 151,589
236752,0,0.0,475,11
85759,0,0.0,1 82,236
46203,0,0.0,64,171
167119,0,0.0,1,
104653,0,0.0,84,1
49163,0,0.0,4,


0.0 <= volumes_delta <= 0.3131868131868132


,duplicates,volumes_delta,volumes_x,volumes_y
28086,0,0.916667,1 82,1 2
152652,0,0.933333,1 412,1 421
153160,0,0.933333,1 221,1 21
107548,0,0.916667,1 82,1 2
79676,0,0.916667,1 82,1 2
226931,0,0.916667,191,1 91
247531,0,0.933333,1 45,1 415
7921,0,0.916667,1 82,1 2
7924,0,0.916667,1 82,1 2
250505,0,0.933333,1 32,1 392


0.9166666666666666 <= volumes_delta <= 0.9333333333333332


,duplicates,volumes_delta,volumes_x,volumes_y
158545,0,0.916667,1 82,1 2
258762,0,0.916667,1 82,1 2
250461,0,0.916667,1 32,1 2
320,1,0.904762,1 169,1 0 169
86200,0,0.916667,1 82,1 2
250460,0,0.916667,1 32,1 2
135609,0,0.916667,191,1 91
13,1,0.904762,1 169,1 0 169
107545,0,0.916667,1 82,1 2
165069,0,0.916667,1 82,1 2


0.9047619047619048 <= volumes_delta <= 0.9166666666666666


Attribute $\texttt{volumes}$ holds rows with missing data. The data pairs with missing values will be marked with a special negative value.

In [75]:
df_feature_base = dpf.mark_missing(df_feature_base, attribute, factor)

## DataFrame with Attributes and Similarity Features

The metric for each attribute of the feature DataFrame has been decided and the similarity features have been calculated. In this last step, the columns of the DataFrame are reordered in order to place the $\_\texttt{delta}$ columns close to their input origins $\_\texttt{x}$ and $\_\texttt{y}$ and some sample records are shown.

In [76]:
# Take _x, _y, and _delta columns together
fb_col_list = df_feature_base.columns.tolist()
fb_col_list.sort()
# Move target column to first place
fb_col_list.insert(0, fb_col_list.pop(fb_col_list.index('duplicates')))
# Reorder DataFrame columns
df_attribute_with_sim_feature = pd.DataFrame(df_feature_base, columns=fb_col_list)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_attribute_with_sim_feature.columns)

class_label = ['uniques', 'duplicate']

for i in class_label:
    display(df_attribute_with_sim_feature[df_attribute_with_sim_feature.duplicates==class_label.index(i)].sample(n=10))
    print(i)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y,coordinate_N_delta,coordinate_N_x,coordinate_N_y,corporate_full_delta,corporate_full_x,corporate_full_y,doi_delta,doi_x,doi_y,edition_delta,edition_x,edition_y,exactDate_delta,exactDate_x,exactDate_y,format_postfix_delta,format_postfix_x,format_postfix_y,format_prefix_delta,format_prefix_x,format_prefix_y,isbn_delta,isbn_x,isbn_y,ismn_delta,ismn_x,ismn_y,musicid_delta,musicid_x,musicid_y,part_delta,part_x,part_y,person_100_delta,person_100_x,person_100_y,person_245c_delta,person_245c_x,person_245c_y,person_700_delta,person_700_x,person_700_y,pubinit_delta,pubinit_x,pubinit_y,scale_delta,scale_x,scale_y,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y,ttlfull_246_delta,ttlfull_246_x,ttlfull_246_y,volumes_delta,volumes_x,volumes_y
112594,0,-0.10,,,-0.10,,,-0.10,,,-0.10,,,-0.05,,4,0.250,2009aaaa,1991uuuu,0.428571,020000,030000,0.0,bk,vm,0.0,[978-3-15-020008-7],[],-0.1,,,-0.10,,,0.000000,20008,3,-0.050000,austenjane,,0.603305,jane austen ; aus dem engl. übers. von ursula ...,sigrid kessler... [et al.] ; [éd.:] interkanto...,-0.100000,,,0.626984,reclam,staatlicher lehrmittelverlag,-0.10,,,0.509009,emma,"bonne chance!, cours de langue française, deux...",-0.10,,,0.000000,600,256
62281,0,-0.10,,,-0.10,,,-0.10,,,-0.10,,,-0.10,,,0.500,18aaaaaa,1956uuuu,0.428571,010200,010100,1.0,mu,mu,1.0,[],[],-0.1,,,-0.05,,5714,-0.100000,,,1.000000,mozartwolfgang amadeus,mozartwolfgang amadeus,0.627285,klavierauszug revidiert von gustav f. kogel ; ...,w. a. mozart ; nach dem in der deutschen staat...,0.615950,"kogelgustav friedrich, vogelemil","zallingermeinhard von, mozartwolfgang amadeus",-0.050000,,peters,-0.10,,,0.882540,"die zauberflöte, oper in zwei akten","die zauberflöte, oper in 2 aufzügen",-0.10,,,-0.050000,,1 234
46078,0,-0.10,,,-0.10,,,-0.10,,,-0.10,,,-0.10,,,0.250,2005aaaa,1998uuuu,1.000000,020000,020000,1.0,bk,bk,0.0,[3-8067-5097-1],[],-0.1,,,-0.10,,,-0.050000,,115,0.566061,jacquetluc,mozartwolfgang amadeus,0.479720,luc jacquet,justus lustig,-0.050000,,lustigjustus,-0.050000,gerstenberg,,-0.10,,,0.614935,die reise der pinguine,"die zauberflöte, frei nach emanuel schikaneder...",-0.10,,,0.666667,64,24
215937,0,-0.10,,,-0.10,,,-0.05,suisse,,-0.10,,,1.00,6,6,0.750,20159999,2015uuuu,1.000000,020000,020000,1.0,bk,bk,0.0,[978-3-7190-3171-8],"[978-3-13-128546-1, 3-13-128546-X]",-0.1,,,-0.10,,,-0.100000,,,-0.050000,,möllerhans-jürgen,0.501227,hrsg. heinrich honsell ... [et al.],"hans-jürgen möller, gerd laux, arno deister ; ...",0.514722,honsellheinrich,"lauxgerd, deisterarno, schulte-körnegerd",-0.050000,helbing lichtenhahn,,-0.10,,,0.518627,obligationenrecht,"psychiatrie, psychosomatik und psychotherapie",-0.05,or,,0.000000,1,670
184692,0,-0.05,e0074147,,-0.05,n0460833,,-0.05,schweizeidgenössisches topographisches bureau,,-0.10,,,-0.05,1863,,0.250,1862aaaa,2006uuuu,0.111111,010300,020053,0.0,mp,bk,0.0,[],"[978-3-598-31798-9 (print), 978-3-11-096277-2]",-0.1,,,-0.10,,,0.702564,23 23 1863 23,38 38,-0.050000,,mortzfeldpeter,0.500661,g. h. dufour direxit ; h. müllhaupt sculpsit,"mortzfeld, peter; raabe, paul",0.506759,"dufourguillaume-henri, müllhauptheinrich, koeg...",raabepaul,0.594444,[eidg. topographisches bureau],de gruyter saur,-0.05,100000,,0.519083,"domo d'ossola, arona",katalog der graphischen porträts in der herzog...,-0.05,"[domodossola, arona]",,0.733333,1,1 450
92974,0,-0.10,,,-0.10,,,-0.10,,,-0.10,,,-0.10,,,0.250,1843aaaa,2002uuuu,0.111111,020000,030100,0.0,bk,vm,0.0,[],"[3-292-00235-4 (Livre de l'élève, Cahier d'exe...",-0.1,,,-0.10,,,-0.050000,,3,-0.100000,,,0.646408,[emanuel schikaneder] ; in musik gesetzt von w...,"[autorinnen und autoren: sigrid kessler, brigi...",0.519027,mozartwolfgang amadeus,"kesslersigrid, suterbrigitte, waltherrudolf",0.447370,"orell, füssli dr.",berner lehrmittel- und medienverlag,-0.10,,,0.569264,"die zauberflöte, grosse oper in zwei akten","bonne chance!, version realschule",-0.10,,,0.833333,31,3
127910,0,-0.10,,,-0.10,,,-0.10,,,-0.10,,,-0.10,,,

uniques


,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y,coordinate_N_delta,coordinate_N_x,coordinate_N_y,corporate_full_delta,corporate_full_x,corporate_full_y,doi_delta,doi_x,doi_y,edition_delta,edition_x,edition_y,exactDate_delta,exactDate_x,exactDate_y,format_postfix_delta,format_postfix_x,format_postfix_y,format_prefix_delta,format_prefix_x,format_prefix_y,isbn_delta,isbn_x,isbn_y,ismn_delta,ismn_x,ismn_y,musicid_delta,musicid_x,musicid_y,part_delta,part_x,part_y,person_100_delta,person_100_x,person_100_y,person_245c_delta,person_245c_x,person_245c_y,person_700_delta,person_700_x,person_700_y,pubinit_delta,pubinit_x,pubinit_y,scale_delta,scale_x,scale_y,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y,ttlfull_246_delta,ttlfull_246_x,ttlfull_246_y,volumes_delta,volumes_x,volumes_y
270,1,-0.1,,,-0.1,,,-0.10,,,-0.1,,,-0.1,,,0.75,2007aaaa,2007uuuu,1.0,010300,010300,1.0,vm,vm,1.0,[],[],-0.1,,,1.00,502023,502023,-0.10,,,-0.1,,,0.693101,ein film von volker schlöndorff ; nach dem rom...,regie: volker schlöndorff ; drehbuch: volker s...,0.610982,"frischmax, schlöndorffvolker","schlöndorffvolker, wurlitzerrudy, frischmax",0.947917,kinowelt home entertainment,kinowelt home entertainment gmbh,-0.1,,,1.000000,homo faber,homo faber,-0.1,,,1.000000,2 109,2 109
544,1,-0.1,,,-0.1,,,-0.10,,,-0.1,,,-0.1,,,0.75,2007aaaa,2007uuuu,1.0,010200,010200,1.0,mu,mu,1.0,[],[],0.0,m006450510,9790006450510,1.00,4553,4553,-0.10,,,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus,0.696786,wolfgang amadeus mozart ; libretto: emanuel sc...,w.a. mozart ; libretto von emanuel schikenader...,-0.050000,,"schelhassmartin, schikanederemanuel, mozartwol...",-0.050000,,bärenreiter,-0.1,,,0.734189,"die zauberflöte, eine deutsche oper in zwei au...","die zauberflöte, the magic flute : a german op...",-0.1,,,0.733333,1,1 245
153,1,-0.1,,,-0.1,,,-0.05,"opernhaus (zürich)orchester, opernhaus (zürich...",,-0.1,,,-0.1,,,0.75,1988aaaa,1988uuuu,1.0,040100,040100,1.0,mu,mu,1.0,[],[],-0.1,,,-0.05,242,,-0.10,,,1.0,mozartwolfgang amadeus,mozartwolfgang amadeus,0.720721,mozart ; [livret emanuel schikaneder],mozart,0.897436,"harnoncourtnikolaus, schikanederemanuel",harnoncourtnikolaus,1.000000,teldec,teldec,-0.1,,,0.884058,"die zauberflöte, kv 620",die zauberflöte,-0.1,,,0.692308,2 73 17 70 26,2
1296,1,-0.1,,,-0.1,,,1.00,schweiz,schweiz,-0.1,,,1.0,6,6,1.00,20159999,20159999,1.0,020000,020000,1.0,bk,bk,1.0,[978-3-7190-3171-8 (Bd. 1)],[978-3-7190-3171-8 (Bd. 1)],-0.1,,,-0.10,,,-0.10,,,-0.1,,,1.000000,hrsg.: heinrich honsell ... [et al.],hrsg.: heinrich honsell ... [et al.],1.000000,"honsellheinrich, ammanncaterina","honsellheinrich, ammanncaterina",1.000000,helbing lichtenhahn,helbing lichtenhahn,-0.1,,,1.000000,obligationenrecht,obligationenrecht,1.0,or,or,-0.100000,,
286,1,-0.1,,,-0.1,,,-0.10,,,-0.1,,,-0.1,,,1.00,20071990,20071990,1.0,010300,010300,1.0,vm,vm,1.0,[],[],-0.1,,,-0.05,502023,,-0.10,,,-0.1,,,0.668770,ein filme von volker schlöndorff,ein volker schlöndorff film ; nach dem gleichn...,0.973134,"schlöndorffvolker, frischmax, delpyjulie, shep...","schlöndorffvolker, frischmax, shepardsam, delp...",-0.100000,,,-0.1,,,1.000000,homo faber,homo faber,-0.1,,,1.000000,2 109,2 109
1002,1,-0.1,,,-0.1,,,-0.10,,,-0.1,,,1.0,3,3,0.75,2001aaaa,2001uuuu,1.0,020000,020000,1.0,bk,bk,1.0,[3-13-127283-X],[3-13-127283-X],-0.1,,,-0.10,,,-0.10,,,1.0,schusterhans-peter,schusterhans-peter,1.000000,"hans-peter schuster, hans-joachim trappe","hans-peter schuster, hans-joachim trappe",1.000000,trappehans-joachim,trappehans-joachim,1.000000,g. thieme,g. thieme,-0.1,,,1.000000,ekg-kurs für isabel,ekg-kurs für isabel,-0.1,,,1.000000,323,323
133,1,-0.1,,,-0.1,,,-0.10,,,-0.1,,,-0.1,,,0.75,2007aaaa,2007uuuu,1.0,020000,020000,1.0,bk,bk,1.0,"[978-3-7815-1531-4, 3-7815-1531-1]",[978-3-7815-1531-4],-0.1,,,-0.10,,,-0.10,,,-0.1,,,0.864546,hrsg. von michaela gläser-zikuda und tina hascher,hrsg. von michaela gläser-zikuda ... [et al.],0.923529,"gläser-zikudamichaela, haschertina",gläser-zikudamichaela,-0.050000

duplicate


## Summary

This chapter covers the central area of feature construction. The features of the feature matrix have been generated for each attribute of Swissbib's raw data, deciding on its similarity metric. With these metric values, the feature base DataFrame has been extended and a new DataFrame with the attribute values of the pairs together with their calculated similarity value have been generated. The similarity values will be the final features for training and performance testing of the models, compare [[JudACaps](./A_References.ipynb#judacaps)].

In [77]:
columns_metadata_dict['similarity_metrics']

{'coordinate_E': LCSStr({'qval': 1, 'external': True}),
 'coordinate_N': LCSStr({'qval': 1, 'external': True}),
 'corporate_full': LCSStr({'qval': 1, 'external': True}),
 'doi': Identity({'qval': 1, 'external': True}),
 'edition': Jaccard({'qval': 1, 'as_set': False, 'external': True}),
 'exactDate': Hamming({'qval': 1, 'test_func': <function Base._ident at 0x1087df510>, 'truncate': False, 'external': True}),
 'format_prefix': Identity({'qval': 1, 'external': True}),
 'format_postfix': Jaccard({'qval': 2, 'as_set': False, 'external': True}),
 'isbn': Identity({'qval': 1, 'external': True}),
 'ismn': Identity({'qval': 1, 'external': True}),
 'musicid': LCSStr({'qval': 1, 'external': True}),
 'part': StrCmp95({'long_strings': False, 'external': True}),
 'person_100': StrCmp95({'long_strings': False, 'external': True}),
 'person_700': StrCmp95({'long_strings': False, 'external': True}),
 'person_245c': Jaro({'qval': 1, 'long_tolerance': False, 'winklerize': False, 'external': True}),
 'pu

The similarity metric decided for each attribute has been added as an additional kind of information to the columns metadata dictionary. The following table gives this summary in a structured form and lists the metric used for each attribute. Attributes with the same font color indicate similar types of values (description column) for better orientation.

| attribute     | subtype | description | similarity metric |
| ------------- |:--------|:------------|:------------------|
|<font color='red'>[coordinate](#coordinate)</font>|<font color='red'>\_E</font>|<font color='red'>Code(9)</font>|<font color='red'>LCSStr</font>|
|               |<font color='red'>\_N</font>|<font color='red'>Code(9)</font>|<font color='red'>LCSStr</font>|
|<font color='blue'>[corporate](#corporate)</font>|<font color='blue'>\_full</font>|<font color='blue'>Name</font>|<font color='blue'>LCSStr</font>|
|<font color='green'>[doi](#doi)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='orange'>[edition](#edition)</font>|         |<font color='orange'>Number</font>|<font color='orange'>Jaccard</font>|
|<font color='black'>[exactDate](#exactDate)</font>|         |<font color='black'>Date</font>|<font color='black'>Hamming</font>|
|<font color='red'>[format](#format)</font>|<font color='red'>\_prefix</font>|<font color='red'>Code(2)</font>|<font color='red'>Identity</font>|
|               |<font color='red'>\_postfix</font>|<font color='red'>Code(6)</font>|<font color='red'>Jaccard (qval=2)</font>|
|<font color='green'>[isbn](#isbn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[ismn](#ismn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[musicid](#musicid)</font>|         |<font color='green'>Identifier</font>|<font color='green'>LCSStr</font>|
|<font color='orange'>[part](#part)</font>|         |<font color='orange'>Number</font>|<font color='orange'>StrCmp95</font>|
|<font color='blue'>[person](#person)</font>|<font color='blue'>\_100</font>|<font color='blue'>Name</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_700</font>|<font color='blue'>Name</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_245c</font>|<font color='blue'>Name</font>|<font color='blue'>Jaro</font>|
|<font color='blue'>[pubinit](#pubinit)</font>|         |<font color='blue'>Name</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[scale](#scale)</font>|         |<font color='orange'>Number</font>|<font color='orange'>Jaccard</font>|
|<font color='blue'>[ttlfull](#ttlfull)</font>|<font color='blue'>\_245</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|               |<font color='blue'>\_246</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[volumes](#volumes)</font>|         |<font color='orange'>Number</font>|<font color='orange'>StrCmp95</font>|

### Full Feature Matrix with Target Vector Handover

To hand over the resulting DataFrame of this chapter, the DataFrame is saved into a pickle file that will be read in the next chapters [Features Discussion and Dummy Classifier Baseline](./5_FeatureDiscussionDummyBaseline.ipynb) as input.

In [78]:
import pickle as pk

# Binary intermediary files
with open(os.path.join(path_goldstandard,
                       'labelled_feature_matrix_full.pkl'), 'wb') as df_output_file:
    pk.dump(df_attribute_with_sim_feature, df_output_file)

# Binary intermediary DataFrame file for docid's
with open(os.path.join(path_goldstandard, 'index_docids_df.pkl'), 'wb') as df_output_file:
    pk.dump(df_index_docids, df_output_file)

Also the full metadata dictionary is to be persisted for handover to subsequent chapters.

In [79]:
# The target is still needed for the feature matrix
columns_metadata_dict['features'].append('duplicates')

for k in columns_metadata_dict.keys():
    print(k, '\n', columns_metadata_dict[k], '\n')

data_analysis_columns 
 ['coordinate_E', 'coordinate_N', 'corporate_full', 'doi', 'edition', 'exactDate', 'format_prefix', 'format_postfix', 'isbn', 'ismn', 'musicid', 'part', 'person_100', 'person_700', 'person_245c', 'pubinit', 'scale', 'ttlfull_245', 'ttlfull_246', 'volumes'] 

columns_to_use 
 ['duplicates', 'coordinate_E_x', 'coordinate_E_y', 'coordinate_N_x', 'coordinate_N_y', 'corporate_full_x', 'corporate_full_y', 'doi_x', 'doi_y', 'edition_x', 'edition_y', 'exactDate_x', 'exactDate_y', 'format_prefix_x', 'format_prefix_y', 'format_postfix_x', 'format_postfix_y', 'isbn_x', 'isbn_y', 'ismn_x', 'ismn_y', 'musicid_x', 'musicid_y', 'part_x', 'part_y', 'person_100_x', 'person_100_y', 'person_700_x', 'person_700_y', 'person_245c_x', 'person_245c_y', 'pubinit_x', 'pubinit_y', 'scale_x', 'scale_y', 'ttlfull_245_x', 'ttlfull_245_y', 'ttlfull_246_x', 'ttlfull_246_y', 'volumes_x', 'volumes_y'] 

similarity_metrics 
 {'coordinate_E': LCSStr({'qval': 1, 'external': True}), 'coordinate_N': L

In [80]:
# Binary intermediary metadata file
with open(os.path.join(path_goldstandard,
                       'columns_metadata.pkl'), 'wb') as dict_output_file:
    pk.dump(columns_metadata_dict, dict_output_file)